In [1]:
import os
import shutil
import imageio
import numpy as np

from tqdm.notebook import tqdm

from PIL import Image
from mytools import Video, get_cuts

# Extract cuts and noncuts from videos

## Prepare directories for data.

In [2]:
path_data = 'data'
path_cuts = os.path.join(path_data, 'cuts')
path_noncuts = os.path.join(path_data, 'noncuts')

if os.path.exists(path_data):
    shutil.rmtree(path_data)
if os.path.exists(path_cuts):
    shutil.rmtree(path_cuts)
if os.path.exists(path_noncuts):
    shutil.rmtree(path_noncuts)

os.makedirs(path_data)
os.makedirs(path_cuts)
os.makedirs(path_noncuts)

## Extracting cuts from videos

In [3]:
output_size = (640, 360)
output_size = (320, 180)
output_size = (160, 90)
output_size

(160, 90)

### Video 0

In [4]:
path_timeline0 = 'timelines/0.kdenlive'
path_video0 = 'videos/0.mp4'

In [5]:
video0 = Video(path_video0)
cuts0 = get_cuts(path_timeline0)

print(f'video0.length = {video0.length}')

video0.length = 5875


In [6]:
video = video0
cuts = cuts0
filename_template = '0-{0}.gif'

for i in tqdm(range(len(cuts)), total=len(cuts)):
    filename = filename_template.format(i)
    frames = video.get_matrix(cuts[i]-2, cuts[i]+2)
    images = [Image.fromarray(frame).resize(output_size) for frame in frames]
    imageio.mimsave(os.path.join(path_cuts, filename), images, fps=8)
    

  0%|          | 0/151 [00:00<?, ?it/s]

### Video 1

In [7]:
path_timeline1 = 'timelines/1.kdenlive'
path_video1 = 'videos/1.mp4'

In [8]:
video1 = Video(path_video1)
cuts1 = get_cuts(path_timeline1)

print(f'video1.length = {video1.length}')

video1.length = 5750


In [9]:
video = video1
cuts = cuts1
filename_template = '1-{0}.gif'

for i in tqdm(range(len(cuts)), total=len(cuts)):
    filename = filename_template.format(i)
    frames = video.get_matrix(cuts[i]-2, cuts[i]+2)
    images = [Image.fromarray(frame).resize(output_size) for frame in frames]
    imageio.mimsave(os.path.join(path_cuts, filename), images, fps=8)
    

  0%|          | 0/240 [00:00<?, ?it/s]

In [10]:
print(f"There are {len(os.listdir(path_cuts))} real cut examples extracted to {path_cuts}")

There are 391 real cut examples extracted to data/cuts


## Extracting noncuts from videos

In [11]:
noncut_inputs0 = np.unique(np.concatenate([[0, video0.length], cuts0]))
noncut_inputs0 = [np.arange(noncut_inputs0[i], noncut_inputs0[i+1]-3) for i in range(len(noncut_inputs0)-1)]
noncut_inputs0 = np.concatenate(noncut_inputs0)
print(f'There are {len(noncut_inputs0)} possible noncat inputs for video1.')

There are 5419 possible noncat inputs for video1.


In [12]:
noncut_inputs1 = np.unique(np.concatenate([[0, video1.length], cuts1]))
noncut_inputs1 = [np.arange(noncut_inputs1[i], noncut_inputs1[i+1]-3) for i in range(len(noncut_inputs1)-1)]
noncut_inputs1 = np.concatenate(noncut_inputs1)
print(f'There are {len(noncut_inputs1)} possible noncat inputs for video1.')

There are 5047 possible noncat inputs for video1.


In [13]:
number_of_cuts = len(cuts0) + len(cuts1)
number_of_noncuts = len(noncut_inputs0) + len(noncut_inputs1)
print('Portion of cuts is {0:.2f}%'.format(100*number_of_cuts/(number_of_noncuts + number_of_cuts)))

Portion of cuts is 3.60%


In [14]:
noncut_use_portion = 0.4
noncut_inputs_use0 = np.sort(np.random.choice(noncut_inputs0, int(noncut_use_portion*len(noncut_inputs0)), replace=False))
noncut_inputs_use1 = np.sort(np.random.choice(noncut_inputs1, int(noncut_use_portion*len(noncut_inputs1)), replace=False))

print(f'I will use {len(noncut_inputs_use0) + len(noncut_inputs_use1)} noncuts.')

I will use 4185 noncuts.


Can extract more, using `noncut_use_portion` variable. If `noncut_use_portion = 1` extract all.

In [15]:
noncut_inputs = noncut_inputs_use0
video = video0
filename_template = '0-{0}.gif'

for i in tqdm(range(len(noncut_inputs)), total=len(noncut_inputs)):
    filename = filename_template.format(i)
    frames = video.get_matrix(noncut_inputs[i], noncut_inputs[i]+4)
    images = [Image.fromarray(frame).resize(output_size) for frame in frames]
    imageio.mimsave(os.path.join(path_noncuts, filename), images, fps=8)

  0%|          | 0/2167 [00:00<?, ?it/s]

In [16]:
noncut_inputs = noncut_inputs_use1
video = video1
filename_template = '1-{0}.gif'

for i in tqdm(range(len(noncut_inputs)), total=len(noncut_inputs)):
    filename = filename_template.format(i)
    frames = video.get_matrix(noncut_inputs[i], noncut_inputs[i]+4)
    images = [Image.fromarray(frame).resize(output_size) for frame in frames]
    imageio.mimsave(os.path.join(path_noncuts, filename), images, fps=8)

  0%|          | 0/2018 [00:00<?, ?it/s]

# Split Data

In [17]:
path_validation = os.path.join(path_data, 'validation')
path_validation_cuts = os.path.join(path_validation, 'cuts')
path_validation_noncuts = os.path.join(path_validation, 'noncuts')

path_training = os.path.join(path_data, 'training')
path_training_cuts = os.path.join(path_training, 'cuts')
path_training_noncuts = os.path.join(path_training, 'noncuts')

if os.path.exists(path_validation):
    shutil.rmtree(path_validation)
if os.path.exists(path_training):
    shutil.rmtree(path_training)
    
os.makedirs(path_validation)
os.makedirs(path_validation_cuts)
os.makedirs(path_validation_noncuts)

os.makedirs(path_training)
os.makedirs(path_training_cuts)
os.makedirs(path_training_noncuts)

In [18]:
split_parameter = 0.8

In [19]:
filenames_cuts = os.listdir(path_cuts)
np.random.shuffle(filenames_cuts)

filenames_cuts_training = filenames_cuts[:int(split_parameter*len(filenames_cuts))]
filenames_cuts_validation = filenames_cuts[int(split_parameter*len(filenames_cuts)):]

print('Copy training cuts:')
for filename in tqdm(filenames_cuts_training, total=len(filenames_cuts_training)):
    shutil.copyfile(os.path.join(path_cuts, filename), os.path.join(path_training_cuts, filename))
    
print('Copy validation cuts:')
for filename in tqdm(filenames_cuts_validation, total=len(filenames_cuts_validation)):
    shutil.copyfile(os.path.join(path_cuts, filename), os.path.join(path_validation_cuts, filename))

Copy training cuts:


  0%|          | 0/312 [00:00<?, ?it/s]

Copy validation cuts:


  0%|          | 0/79 [00:00<?, ?it/s]

In [20]:
filenames_noncuts = os.listdir(path_noncuts)
np.random.shuffle(filenames_noncuts)

filenames_noncuts_training = filenames_noncuts[:int(split_parameter*len(filenames_noncuts))]
filenames_noncuts_validation = filenames_noncuts[int(split_parameter*len(filenames_noncuts)):]

print('Copy training noncuts:')
for filename in tqdm(filenames_noncuts_training, total=len(filenames_noncuts_training)):
    shutil.copyfile(os.path.join(path_noncuts, filename), os.path.join(path_training_noncuts, filename))
    
print('Copy validation noncuts:')
for filename in tqdm(filenames_noncuts_validation, total=len(filenames_noncuts_validation)):
    shutil.copyfile(os.path.join(path_noncuts, filename), os.path.join(path_validation_noncuts, filename))

Copy training noncuts:


  0%|          | 0/3348 [00:00<?, ?it/s]

Copy validation noncuts:


  0%|          | 0/837 [00:00<?, ?it/s]

# Augmentation

## Augmentation Cuts

In [21]:
portion_augmentated = 0.73

number_training_cuts = len(os.listdir(path_training_cuts))
number_training_noncuts = len(os.listdir(path_training_noncuts))

number_augmentated = int(number_training_cuts*portion_augmentated/(1 - portion_augmentated))

print(f'I am going to create {number_augmentated} augmentated cuts.')
print('Then portion of cuts in training_data will be {0:.2f} %'.format(100*(number_augmentated + number_training_cuts)/(number_augmentated + number_training_cuts + number_training_noncuts)))

I am going to create 843 augmentated cuts.
Then portion of cuts in training_data will be 25.65 %


In [22]:
filename_template = 'a-{0}.gif'
videos = [video0, video1]
inputs = [noncut_inputs0, noncut_inputs1]

for i in tqdm(range(number_augmentated)):
    filename = filename_template.format(i)
    
    id0 = np.random.randint(2)
    id1 = np.random.randint(2)
    input0 = np.random.choice(inputs[id0])
    input1 = np.random.choice(inputs[id1])
    
    
    
    while (input1 == input0 + 2) and (id0 == id1):
        # Change if noncut randomly got
        print('noncut randomly got once')
        id0 = np.random.randint(2)
        id1 = np.random.randint(2)
        input0 = np.random.choice(inputs[id0])
        input1 = np.random.choice(inputs[id1])
    
    frames0 = videos[id0].get_matrix(input0, input0 + 2)
    frames1 = videos[id1].get_matrix(input1, input1 + 2)
    frames = np.concatenate([frames0, frames1])
    
    images = [Image.fromarray(frame).resize(output_size) for frame in frames]
    imageio.mimsave(os.path.join(path_training_cuts, filename), images, fps=8)

  0%|          | 0/843 [00:00<?, ?it/s]

I built an augmentation for cuts. But there are 2 problems:

1. I will need to upload gif every iteration, if I'll use that. And this is not good, because that can be too long.
2. There are not realy many situations.

The first problem can be solved, by developing generator from videos and timelines. I will download each video only once and then extract cut and noncut moments, using some pd.DataFrame file.

There are few steps how to build that:
1. Add methods to mytools.Video, which will extract images, not arrays.
2. Develop a table format to store a cut and noncut info
3. Develop a function or class, which will extract array shape (4, width, height, 3) from given row
4. Develop a generator, which will use a function or class from 3

The 2nd problem can be easilly solved by adding flips, zooms and reverses to both: cuts and noncuts. But that will be harder if add this to steps 2-4.

The function or class from step 3 can read unique filenames from `video` and `timeline` raws of table from step 1, and then upload them using `mytools.Video`, `mytools.get_timeline` to dictionaries, which keys are this filenames.